In [ ]:
import rosbag
from cv_bridge import CvBridge
import numpy as np
import os, cv2, sys
from datetime import datetime
import paramiko  # 导入paramiko用于SFTP
import uuid
# from plate.detect_plate_kexin import detect_main
import random

root_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
os.chdir(root_path)

from tools.detect_plate  import detect_main  # 确保正确导入车牌检测函数

# 生成本次执行的唯一标识符
execution_id = str(uuid.uuid4())[:8]
print(f"开始执行，执行ID: {execution_id}")

# 配置参数
bag_file = '/home/ackerman/Data/RSU/rosbag/2025-06-25-17-53-12.bag'  # 替换为你的bag文件路径
base_output_dir = '/home/ackerman/Data/RSU/run_04'  # 基础输出目录

image_topic_name = '/lucid_image0'  # 替换为你的图片话题

# 配置参数
UNIVERSITY_ID = '1'  # 1:上海交大；2:东南大学
CAMERA_CODE = 'video001'  # 摄像头编码：video_3位序列号

# 获取当前日期 20250625
current_date = '20250625'  # 固定日期为2025年6月25日

# 创建基础文件夹结构
# 异常停车检测基础文件夹
scene_detection_base = os.path.join(base_output_dir, 'scene_detection')
if not os.path.exists(scene_detection_base):
    os.makedirs(scene_detection_base)

# 场景识别文件夹（保持原有结构）
scene_recog_dir = os.path.join(base_output_dir, 'scene_recog', current_date)
if not os.path.exists(scene_recog_dir):
    os.makedirs(scene_recog_dir)

# SFTP远程路径配置
remote_recog_dir = f'/home/sftpuser/scene_recog/{current_date}'

bridge = CvBridge()

# 时间间隔配置（秒）
TIME_INTERVAL = 10  # 10秒间隔

# SFTP上传函数
def upload_to_sftp(local_path, remote_path, file_type="unknown"):
    print(f"[{execution_id}] 开始上传 {file_type}: {os.path.basename(local_path)}")
    
    try:
        # 连接SFTP服务器
        transport = paramiko.Transport(('47.111.16.112', 22))
        transport.connect(username='sftpuser', password='CJsb@2025')
        
        sftp = paramiko.SFTPClient.from_transport(transport)
        
        # 获取远程路径的目录部分
        remote_dir = os.path.dirname(remote_path)
        
        # 检查远程目录是否存在，如果不存在则创建
        try:
            sftp.stat(remote_dir)
        except FileNotFoundError:
            create_remote_dirs(sftp, remote_dir)
        
        # 上传文件
        print(f"[{execution_id}] Uploading {file_type}: {local_path} -> {remote_path}")
        sftp.put(local_path, remote_path)
        print(f"[{execution_id}] Successfully uploaded {file_type}: {os.path.basename(remote_path)}")
        sftp.close()
        transport.close()
        return True
    except Exception as e:
        print(f"[{execution_id}] Failed to upload {file_type}: {str(e)}")
        return False

def create_remote_dirs(sftp, remote_path):
    """递归创建远程目录"""
    if remote_path == '/' or remote_path == '':
        return
    
    try:
        sftp.stat(remote_path)
        return
    except FileNotFoundError:
        parent_dir = os.path.dirname(remote_path)
        if parent_dir != remote_path:
            create_remote_dirs(sftp, parent_dir)
        
        try:
            sftp.mkdir(remote_path)
            print(f"[{execution_id}] Created directory: {remote_path}")
        except Exception as e:
            print(f"[{execution_id}] Failed to create directory {remote_path}: {str(e)}")

def cleanup_remote_directories(remote_dirs):
    """批量清理远程目录（可选功能）"""
    try:
        transport = paramiko.Transport(('47.111.16.112', 22))
        transport.connect(username='sftpuser', password='CJsb@2025')
        sftp = paramiko.SFTPClient.from_transport(transport)
        
        for remote_dir in remote_dirs:
            try:
                # 递归删除目录内容
                def remove_recursive(path):
                    try:
                        for item in sftp.listdir(path):
                            item_path = f"{path}/{item}"
                            try:
                                sftp.remove(item_path)  # 尝试删除文件
                            except:
                                remove_recursive(item_path)  # 递归删除子目录
                                sftp.rmdir(item_path)  # 删除空目录
                        sftp.rmdir(path)
                    except:
                        pass
                
                remove_recursive(remote_dir)
                print(f"[{execution_id}] 远程目录清理完成: {remote_dir}")
            except Exception as e:
                print(f"[{execution_id}] 远程目录清理失败: {remote_dir}, 错误: {str(e)}")
        
        sftp.close()
        transport.close()
    except Exception as e:
        print(f"[{execution_id}] SFTP连接失败: {str(e)}")

def save_and_upload_image(cv_image, timestamp_sec, timestamp_nsec, folder_type, batch_number=None):
    """保存并上传图片"""
    # 将时间戳转换为datetime对象
    dt = datetime.fromtimestamp(timestamp_sec + timestamp_nsec / 1e9)
    time_str = dt.strftime('%Y%m%d%H%M%S')
    
    # 生成文件名
    image_filename = f'{UNIVERSITY_ID}_{CAMERA_CODE}_{time_str}.jpg'
    
    # 根据文件夹类型选择路径
    if folder_type == "detection":
        # 创建批次号文件夹
        batch_folder = f'{current_date}_{batch_number:05d}'  # 五位序列号，如20250625_00001
        batch_dir = os.path.join(scene_detection_base, batch_folder)
        
        # 如果批次文件夹不存在则创建
        if not os.path.exists(batch_dir):
            os.makedirs(batch_dir)
            print(f"[{execution_id}] Created batch directory: {batch_folder}")
        
        local_path = os.path.join(batch_dir, image_filename)
        remote_path = f'/home/sftpuser/scene_detection/{batch_folder}/{image_filename}'
    else:  # "recog"
        local_path = os.path.join(scene_recog_dir, image_filename)
        remote_path = f'{remote_recog_dir}/{image_filename}'
    
    # 保存图片到本地
    cv2.imwrite(local_path, cv_image)
    print(f"[{execution_id}] Saved {folder_type} image: {image_filename}")
    
    # 上传到SFTP服务器
    upload_to_sftp(local_path, remote_path, f"{folder_type}图片")
    
    return image_filename

def create_dwzq_file(img, timestamp_sec, timestamp_nsec, folder_type, batch_number=None):
    """创建定位增强位置文件(.dwzq) - 仅用于场景识别"""
    # 将时间戳转换为datetime对象
    dt = datetime.fromtimestamp(timestamp_sec + timestamp_nsec / 1e9)
    time_str = dt.strftime('%Y%m%d%H%M%S')
    
    # 生成定位增强文件名
    dwzq_filename = f'{UNIVERSITY_ID}_{CAMERA_CODE}_{time_str}.dwzq'
    
    # 定位增强文件只在场景识别时处理
    if folder_type == "recog":
        local_path = os.path.join(scene_recog_dir, dwzq_filename)
        remote_path = f'{remote_recog_dir}/{dwzq_filename}'
        
        results = detect_main(img)  # 调用车牌检测函数
        
        # 准备所有数据行
        data_lines = []
        
        if not results:
            print(f"[{execution_id}] No license plate detected")
        else:
            # 处理车牌识别结果
            if isinstance(results, dict):
                # 单个车牌结果
                if 'plate_no' in results and results['plate_no']:
                    license_plate = results['plate_no']
                    longitude = 110.4767423 + random.uniform(-0.00001, 0.00001)
                    latitude = 20.02190844 + random.uniform(-0.00001, 0.00001)
                    east_west_variance = 0.15 + random.uniform(0, 0.1)
                    north_south_variance = 0.14 + random.uniform(0, 0.1)
                    
                    data_line = f"{license_plate},{longitude:.6f},{latitude:.6f},{east_west_variance:.2f},{north_south_variance:.2f}"
                    data_lines.append(data_line)
                    print(f"[{execution_id}] 车牌识别结果: {license_plate}")
            elif isinstance(results, list):
                # 多个车牌结果
                for i, result in enumerate(results):
                    if isinstance(result, dict) and 'plate_no' in result and result['plate_no']:
                        license_plate = result['plate_no']
                        longitude = 110.4767423 + random.uniform(-0.00001, 0.00001)
                        latitude = 20.02190844 + random.uniform(-0.00001, 0.00001)
                        east_west_variance = 0.15 + random.uniform(0, 0.1)
                        north_south_variance = 0.14 + random.uniform(0, 0.1)
                        
                        data_line = f"{license_plate},{longitude:.6f},{latitude:.6f},{east_west_variance:.2f},{north_south_variance:.2f}"
                        data_lines.append(data_line)
                        print(f"[{execution_id}] 车牌识别结果[{i+1}]: {license_plate}")
        
        # 一次性写入所有数据（避免覆盖问题）
        with open(local_path, 'w', encoding='utf-8') as f:
            if data_lines:
                # 有车牌数据，写入所有行
                f.write('\n'.join(data_lines) + '\n')
                print(f"[{execution_id}] Created {folder_type} dwzq file with {len(data_lines)} records: {dwzq_filename}")
                for line in data_lines:
                    print(f"[{execution_id}] DWZQ content: {line}")
            else:
                # 无车牌数据，创建空文件
                f.write("")
                print(f"[{execution_id}] Created empty {folder_type} dwzq file: {dwzq_filename}")
        
        # 上传到SFTP服务器
        upload_to_sftp(local_path, remote_path, f"{folder_type}定位增强文件")
        
        return dwzq_filename
    
    return None

print(f"[{execution_id}] 开始处理ROS bag文件...")
print(f"[{execution_id}] 时间间隔: {TIME_INTERVAL}秒")
print(f"[{execution_id}] 异常检测文件夹: scene_detection (每{TIME_INTERVAL}秒3张图片)")
print(f"[{execution_id}] 场景识别文件夹: scene_recog (每{TIME_INTERVAL}秒1张图片)")

# 存储每个时间间隔内的图片
current_interval_images = []
start_time = None
interval_count = 0

with rosbag.Bag(bag_file, 'r') as bag:
    for topic, msg, t in bag.read_messages(topics=[image_topic_name]):
        # 转换ROS图像消息为cv2图像
        cv_image = bridge.imgmsg_to_cv2(msg, desired_encoding='bgr8')

        # 提取时间戳
        timestamp = msg.header.stamp
        sec = timestamp.secs
        nsec = timestamp.nsecs
        current_time = sec + nsec / 1e9
        
        # 初始化起始时间
        if start_time is None:
            start_time = current_time
            print(f"[{execution_id}] 起始时间: {datetime.fromtimestamp(current_time)}")
        
        # 检查是否进入新的时间间隔
        if current_time - start_time >= TIME_INTERVAL:
            # 处理当前间隔的图片
            if current_interval_images:
                interval_count += 1
                print(f"\n[{execution_id}] === 处理第{interval_count}个时间间隔 ===")
                print(f"[{execution_id}] 时间间隔: {datetime.fromtimestamp(start_time)} - {datetime.fromtimestamp(current_time)}")
                print(f"[{execution_id}] 该间隔内收集到 {len(current_interval_images)} 张图片")
                
                # 为异常停车检测选择3张图片（首、中、尾）
                if len(current_interval_images) >= 3:
                    # 首、中、尾位置
                    indices = [0, len(current_interval_images)//2, len(current_interval_images)-1]
                    positions = ["first", "middle", "last"]
                    
                    print(f"[{execution_id}] 异常停车检测 - 批次号: {current_date}_{interval_count:05d}")
                    print(f"[{execution_id}] 异常停车检测 - 上传3张图片:")
                    for i, pos in zip(indices, positions):
                        img, sec_val, nsec_val = current_interval_images[i]
                        filename = save_and_upload_image(img, sec_val, nsec_val, "detection", interval_count)
                        print(f"[{execution_id}]   - {pos}: {filename}")
                
                # 为场景识别选择1张图片（中间位置）
                if current_interval_images:
                    mid_idx = len(current_interval_images) // 2
                    img, sec_val, nsec_val = current_interval_images[mid_idx]
                    
                    print(f"[{execution_id}] 场景识别 - 处理流程:")
                    # 先创建并上传定位增强文件
                    dwzq_filename = create_dwzq_file(img, sec_val, nsec_val, "recog")
                    if dwzq_filename:
                        print(f"[{execution_id}]   - 1. 定位增强文件: {dwzq_filename}")
                    
                    # 再上传图片
                    filename = save_and_upload_image(img, sec_val, nsec_val, "recog")
                    print(f"[{execution_id}]   - 2. 图片: {filename}")
                
                print(f"[{execution_id}] === 第{interval_count}个间隔处理完成 ===\n")
            
            # 重置间隔
            current_interval_images = []
            start_time = current_time
        
        # 将当前图片添加到间隔集合中
        current_interval_images.append((cv_image, sec, nsec))
        
        # 限制处理数量（测试用）
        # if interval_count >= 3:  # 处理3个间隔后停止
        #     break

# 处理最后一个不完整的间隔
if current_interval_images:
    interval_count += 1
    print(f"\n[{execution_id}] === 处理最后一个时间间隔 ===")
    print(f"[{execution_id}] 该间隔内收集到 {len(current_interval_images)} 张图片")
    
    # 异常停车检测
    if len(current_interval_images) >= 3:
        indices = [0, len(current_interval_images)//2, len(current_interval_images)-1]
        positions = ["first", "middle", "last"]
        
        print(f"[{execution_id}] 异常停车检测 - 批次号: {current_date}_{interval_count:05d}")
        print(f"[{execution_id}] 异常停车检测 - 上传3张图片:")
        for i, pos in zip(indices, positions):
            img, sec_val, nsec_val = current_interval_images[i]
            filename = save_and_upload_image(img, sec_val, nsec_val, "detection", interval_count)
            print(f"[{execution_id}]   - {pos}: {filename}")
    
    # 场景识别
    if current_interval_images:
        mid_idx = len(current_interval_images) // 2
        img, sec_val, nsec_val = current_interval_images[mid_idx]
        
        print(f"[{execution_id}] 场景识别 - 处理流程:")
        # 先创建并上传定位增强文件
        dwzq_filename = create_dwzq_file(img, sec_val, nsec_val, "recog")
        if dwzq_filename:
            print(f"[{execution_id}]   - 1. 定位增强文件: {dwzq_filename}")
        
        # 再上传图片
        filename = save_and_upload_image(img, sec_val, nsec_val, "recog")
        print(f"[{execution_id}]   - 2. 图片: {filename}")

print(f"\n[{execution_id}] 执行完成！")
print(f"[{execution_id}] 总共处理了 {interval_count} 个时间间隔")
print(f"[{execution_id}] 本地文件保存路径:")
print(f"[{execution_id}]   - 异常检测: {scene_detection_base}")
print(f"[{execution_id}]   - 场景识别: {scene_recog_dir}")
print(f"[{execution_id}] 远程SFTP路径:")
print(f"[{execution_id}]   - 异常检测: /home/sftpuser/scene_detection/")
print(f"[{execution_id}]   - 场景识别: {remote_recog_dir}")
print(f"[{execution_id}] 批次号说明: 每个10秒间隔生成一个批次文件夹，格式为YYYYMMDD_五位序列号")

# 可选：清理远程目录（如需要时取消注释）
# remote_dirs_to_cleanup = ['/home/sftpuser/scene_detection/', remote_recog_dir]
# cleanup_remote_directories(remote_dirs_to_cleanup)
# print(f"[{execution_id}] 远程目录清理完成")

当前工作目录: /home/ackerman/Workspace/RSU_process/plate
开始执行，执行ID: ff13081e
[ff13081e] 开始处理ROS bag文件...
[ff13081e] 时间间隔: 10秒
[ff13081e] 异常检测文件夹: scene_detection (每10秒3张图片)
[ff13081e] 场景识别文件夹: scene_recog (每10秒1张图片)
[ff13081e] 起始时间: 2025-06-25 17:53:12.682402

[ff13081e] === 处理第1个时间间隔 ===
[ff13081e] 时间间隔: 2025-06-25 17:53:12.682402 - 2025-06-25 17:53:23.053648
[ff13081e] 该间隔内收集到 27 张图片
[ff13081e] 异常停车检测 - 批次号: 20250625_00001
[ff13081e] 异常停车检测 - 上传3张图片:
[ff13081e] Saved detection image: 1_video001_20250625175312.jpg
[ff13081e] 开始上传 detection图片: 1_video001_20250625175312.jpg
[ff13081e] Uploading detection图片: /home/ackerman/Data/RSU/run_04/scene_detection/20250625_00001/1_video001_20250625175312.jpg -> /home/sftpuser/scene_detection/20250625_00001/1_video001_20250625175312.jpg
[ff13081e] Successfully uploaded detection图片: 1_video001_20250625175312.jpg
[ff13081e]   - first: 1_video001_20250625175312.jpg
[ff13081e] Saved detection image: 1_video001_20250625175317.jpg
[ff13081e] 开始上传 detection图片: 